In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import itertools

from vivarium import Artifact, InteractiveContext
import ipywidgets
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 8)

This notebook was run on model version 4.5 (built up to the x-factor implementation)

The purpose of this notebook is to investigate the wasting exposure distribution by x-factor exposure at simulation initialization as well as three months later (when many simulants who were initialized into MAM/SAM states will have transitioned to new states). The x-factor value for this notebook was set to 1.5

This notebook was run prior to the implementation of risk-risk correlation in the simulation. Once the correlations are implemented, it is expected that there will be greater correlation between the x-factor and wasting exposure distributions at simulation initialization due to the correlation through birthweight exposure.

In [2]:
path = Path('/ihme/homes/alibow/vivarium_ciff_sam/src/vivarium_ciff_sam/model_specifications/ciff_sam.yaml')
path

PosixPath('/ihme/homes/alibow/vivarium_ciff_sam/src/vivarium_ciff_sam/model_specifications/ciff_sam.yaml')

In [3]:
sim = InteractiveContext(path)

2021-10-11 10:52:41.534 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering metrics.1.population_manager.metrics as modifier to metrics
2021-10-11 10:52:41.536 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:65 - Running simulation from artifact located at /ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf.
2021-10-11 10:52:41.537 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:66 - Artifact base filter terms are ['draw == 0'].
2021-10-11 10:52:41.538 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:67 - Artifact additional filter terms are None.
2021-10-11 10:52:43.090 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline cause_specific_mortality_rate
2021-10-11 10:52:43.091 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline mortality_rate
/ihme/code/alibow/miniconda/envs/vivarium_ciff_sam/lib/python3.6/site-

2021-10-11 10:52:53.590 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering measles.incidence_rate.1.risk_effect.risk_factor.child_wasting.cause.measles.incidence_rate.adjust_target as modifier to measles.incidence_rate
2021-10-11 10:52:53.591 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering measles.incidence_rate.paf.1.lookup_table_38 as modifier to measles.incidence_rate.paf
2021-10-11 10:52:54.174 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering lower_respiratory_infections.incidence_rate.1.risk_effect.risk_factor.child_wasting.cause.lower_respiratory_infections.incidence_rate.adjust_target as modifier to lower_respiratory_infections.incidence_rate
2021-10-11 10:52:54.175 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering lower_respiratory_infections.incidence_rate.paf.1.lookup_table_40 as modifier to lower_respiratory_infections.incidence_rate.paf
2021-10-11 10

2021-10-11 10:53:05.493 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline mild_child_wasting.incidence_rate
2021-10-11 10:53:05.494 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline mild_child_wasting.incidence_rate.paf
2021-10-11 10:53:05.870 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline mild_child_wasting.dwell_time
2021-10-11 10:53:05.872 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline mild_child_wasting.disability_weight
2021-10-11 10:53:05.873 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering disability_weight.4.mild_child_wasting.disability_weight as modifier to disability_weight
2021-10-11 10:53:05.874 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline mild_child_wasting.excess_mortality_rate
2021-10-11 10:53:05.

2021-10-11 10:53:11.625 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering metrics.5.measles_disease_observer.metrics as modifier to metrics
2021-10-11 10:53:11.630 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering metrics.6.lower_respiratory_infections_disease_observer.metrics as modifier to metrics
2021-10-11 10:53:11.635 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering metrics.7.child_wasting_disease_observer.metrics as modifier to metrics
2021-10-11 10:53:11.661 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering metrics.8.categorical_risk_observer.child_stunting.metrics as modifier to metrics
2021-10-11 10:53:11.662 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline metrics


In [4]:
pop0 = sim.get_population()
pop0

,tracked,previous_child_wasting,previous_lower_respiratory_infections,previous_measles,previous_diarrheal_diseases,years_lived_with_disability,years_of_life_lost,cause_of_death,age,sex,...,measles,measles_event_time,measles_event_count,susceptible_to_measles_event_count,susceptible_to_measles_event_time,diarrheal_diseases,diarrheal_diseases_event_time,diarrheal_diseases_event_count,susceptible_to_diarrheal_diseases_event_count,susceptible_to_diarrheal_diseases_event_time
0,True,,,,,0.0,0.0,not_dead,2.761284,Male,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT
1,True,,,,,0.0,0.0,not_dead,2.863263,Male,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT
2,True,,,,,0.0,0.0,not_dead,3.291584,Female,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT
3,True,,,,,0.0,0.0,not_dead,4.900417,Female,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,True,,,,,0.0,0.0,not_dead,3.859252,Female,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT
9997,True,,,,,0.0,0.0,not_dead,0.070874,Female,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT
9998,True,,,,,0.0,0.0,not_dead,4.425916,Female,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT
9999,True,,,,,0.0,0.0,not_dead,4.304582,Male,...,susceptible_to_measles,NaT,0,0,NaT,susceptible_to_diarrheal_diseases,NaT,0,0,NaT


In [5]:
x0 = pd.concat([pop0.loc[:, ['age', 'sex', 'x_factor_propensity','child_wasting']], 
           sim.get_value('x_factor.exposure')(pop0.index)], axis=1)
x0

,age,sex,x_factor_propensity,child_wasting,risk_factor.x_factor_exposure
0,2.761284,Male,0.624358,moderate_acute_malnutrition,cat2
1,2.863263,Male,0.701568,susceptible_to_child_wasting,cat2
2,3.291584,Female,0.287724,susceptible_to_child_wasting,cat2
3,4.900417,Female,0.168520,susceptible_to_child_wasting,cat1
...,...,...,...,...,...
9996,3.859252,Female,0.935051,moderate_acute_malnutrition,cat2
9997,0.070874,Female,0.573722,mild_child_wasting,cat2
9998,4.425916,Female,0.910395,susceptible_to_child_wasting,cat2
9999,4.304582,Male,0.584271,susceptible_to_child_wasting,cat2


In [6]:
pd.crosstab(index=x0['risk_factor.x_factor_exposure'], columns=x0['child_wasting'], normalize='index')

child_wasting,mild_child_wasting,moderate_acute_malnutrition,severe_acute_malnutrition,susceptible_to_child_wasting
risk_factor.x_factor_exposure,,,,
cat1,0.199180,0.084321,0.014585,0.701914
cat2,0.212657,0.077248,0.016398,0.693697


In [7]:
# set simulation run time to three months
# set x-factor to 1.5

sim.run()

# this took about 30-40 minutes

2021-10-11 10:53:14.699 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-01 00:00:00
2021-10-11 10:53:45.792 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-01 12:00:00
2021-10-11 10:54:13.773 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-02 00:00:00
2021-10-11 10:54:41.175 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-02 12:00:00
2021-10-11 10:55:08.685 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-03 00:00:00
2021-10-11 10:55:36.526 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-03 12:00:00
2021-10-11 10:56:01.261 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-04 00:00:00
2021-10-11 10:56:11.744 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-04 12:00:00
2021-10-11 10:56:22.145 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-05 00:00:00
2021-10-11 10:56:32.361 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-05 12:00:00
2021-10-11 10:56:42.666 | DEBUG    | vivarium.framework.engi

2021-10-11 11:09:57.086 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-14 00:00:00
2021-10-11 11:10:07.269 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-14 12:00:00
2021-10-11 11:10:17.663 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-15 00:00:00
2021-10-11 11:10:27.945 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-15 12:00:00
2021-10-11 11:10:38.376 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-16 00:00:00
2021-10-11 11:10:48.693 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-16 12:00:00
2021-10-11 11:10:58.720 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-17 00:00:00
2021-10-11 11:11:08.922 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-17 12:00:00
2021-10-11 11:11:19.066 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-18 00:00:00
2021-10-11 11:11:29.215 | DEBUG    | vivarium.framework.engine:step:142 - 2022-02-18 12:00:00
2021-10-11 11:11:39.060 | DEBUG    | vivarium.framework.engi

2021-10-11 11:24:49.095 | DEBUG    | vivarium.framework.engine:step:142 - 2022-03-30 00:00:00
2021-10-11 11:24:59.420 | DEBUG    | vivarium.framework.engine:step:142 - 2022-03-30 12:00:00
2021-10-11 11:25:09.601 | DEBUG    | vivarium.framework.engine:step:142 - 2022-03-31 00:00:00
2021-10-11 11:25:19.831 | DEBUG    | vivarium.framework.engine:step:142 - 2022-03-31 12:00:00


180

In [8]:
pop3 = sim.get_population()
x3 = pd.concat([pop3.loc[:, ['age', 'sex', 'x_factor_propensity','child_wasting']], 
           sim.get_value('x_factor.exposure')(pop3.index)], axis=1)
x3

,age,sex,x_factor_propensity,child_wasting,risk_factor.x_factor_exposure
0,3.007690,Male,0.624358,mild_child_wasting,cat2
1,3.109670,Male,0.701568,susceptible_to_child_wasting,cat2
2,3.537990,Female,0.287724,susceptible_to_child_wasting,cat2
4,1.043696,Male,0.974205,mild_child_wasting,cat2
...,...,...,...,...,...
10509,0.001165,Male,0.265030,susceptible_to_child_wasting,cat2
10510,0.000425,Male,0.860139,susceptible_to_child_wasting,cat2
10511,0.000247,Female,0.663448,susceptible_to_child_wasting,cat2
10512,0.000818,Male,0.774742,susceptible_to_child_wasting,cat2


In [9]:
pd.crosstab(index=x3['risk_factor.x_factor_exposure'], columns=x3['child_wasting'], normalize='index')


# MAM and SAM exposure is higher among x-factor cat1 (exposed), looks good!!
# a bit curious that mild child wasting is lower among cat1 than cat2, but not incorrect

child_wasting,mild_child_wasting,moderate_acute_malnutrition,severe_acute_malnutrition,susceptible_to_child_wasting
risk_factor.x_factor_exposure,,,,
cat1,0.189671,0.090494,0.027422,0.692413
cat2,0.209902,0.076815,0.016588,0.696695


In [10]:
# MAM prevalence ratio
0.090494/0.076815

# lower than x-factor incidence rate ratio of 1.5
# likely because x-factor exposed simulants are also leaving MAM to SAM at higher rate than unexposed

1.1780771984638418

In [11]:
# SAM prevalence ratio
0.027422/0.016588

# similar, but slightly higher than x-factor incidence rate ratio of 1.5
# exit rate from SAM does not vary by x-factor exposure

1.653122739329636